# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder


In [1]:
import open3d as o3d
import numpy as np
import copy
from scipy.spatial import procrustes
import scipy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, .1, 0])
        target_temp.paint_uniform_color([0, .1, 1])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [3]:
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")
voxel_size = 0.075
# draw_registrations(source, target)

In [4]:
mesh = o3d.io.read_triangle_mesh("global_registration/car1.ply")
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(np.array(mesh.vertices))
draw_registrations(pcd1,pcd1)

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).


### Features

In [5]:
source = source.voxel_down_sample(voxel_size)
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                            max_nn=30))
source_fpfh_features = o3d.pipelines.registration.compute_fpfh_feature(source,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0,max_nn=100))

target = target.voxel_down_sample(voxel_size)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                            max_nn=30))
target_fpfh_features = o3d.pipelines.registration.compute_fpfh_feature(target,
    o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0, max_nn=100))

In [6]:
target_fpfh_features.data.shape, source_fpfh_features.data.shape, target.dimension, source.dimension

((33, 1615),
 (33, 2154),
 <bound method PyCapsule.dimension of PointCloud with 1615 points.>,
 <bound method PyCapsule.dimension of PointCloud with 2154 points.>)

In [16]:
target_points = np.asarray(target.points)
target_centroid = np.mean(target_points, axis=0)
source_points = np.asarray(source.points)
source_centroid = np.mean(source_points, axis=0)

In [49]:
# ICP
target_points.shape, len(target_points), len(source_points)

n = 50
pairs = {}
for ti,tp in enumerate(target_points):
    # mind = np.inf
    # for sp in source_points:
    #     d = np.sqrt( (tp[0]-sp[0])**2 + (tp[1]-sp[1])**2 + (tp[2]-sp[2])**2 )
    #     if d < mind:
    #         mind = d
    # print(f"Optimal: {mind}")

    mind = np.inf
    si = -1
    ri = np.random.randint(0, len(source_points)-1, n)
    for i in ri:
        sp = source_points[i]
        d = np.sqrt( (tp[0]-sp[0])**2 + (tp[1]-sp[1])**2 + (tp[2]-sp[2])**2 )
        if d < mind:
            mind = d
            si = i
    pairs[ti] = si

Cov = target_points[list(pairs.keys())].T @ source_points[list(pairs.values())]
U, S, Vt = np.linalg.svd(Cov)
v = Vt.T
d = np.linalg.det(v @ U.T)
e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])
r = v @ e @ U.T
estimated_rotation = r.T
T = target_centroid - estimated_rotation @ source_centroid
estimated_rotation, T

(array([[ 0.99317008, -0.02667225,  0.11358599],
        [ 0.03071052,  0.99895151, -0.03395214],
        [-0.11256131,  0.03720854,  0.99294787]]),
 array([ 0.38525544, -0.14254391, -0.26332773]))

In [8]:
# data = np.append(target_fpfh_features.data.T,source_fpfh_features.data.T,axis=0)
kd1 = scipy.spatial.KDTree(target_fpfh_features.data.T)
kd2 = scipy.spatial.KDTree(source_fpfh_features.data.T)
indexes = kd1.query_ball_tree(kd2, r=13)
# target => source
matches = {}
for i in range(len(indexes)):
    if indexes[i]:
        matches[i] = indexes[i][0]
n_matched = len(matches)

In [9]:
target_random_sample = np.random.choice(list(matches.keys()), (3,), replace=False)
source_random_sample = [matches[target_random_sample[x]] for x in range(3)]
target_random_sample, source_random_sample

(array([  89,   24, 1614]), [15, 15, 15])

In [10]:
match_targets = target_points[list(matches.keys())]
match_sources = source_points[list(matches.values())]
match_targets.shape == match_sources.shape

True

In [11]:
target_matched_xyz = target_points[target_random_sample] - target_centroid
source_matched_xyz = source_points[source_random_sample] - source_centroid
target_matched_xyz,source_matched_xyz

(array([[ 0.5386475 ,  0.64242538, -0.42536204],
        [-0.55162953, -0.79434884, -0.07560413],
        [ 0.41885583,  0.64024069, -0.42882768]]),
 array([[ 0.25414949,  0.15883807, -0.7056722 ],
        [ 0.25414949,  0.15883807, -0.7056722 ],
        [ 0.25414949,  0.15883807, -0.7056722 ]]))

In [12]:
(estimated_rotation,rmsd) = scipy.spatial.transform.Rotation.align_vectors(target_matched_xyz, source_matched_xyz)
estimated_rotation,rmsd

C:\Users\simut\AppData\Local\Temp\ipykernel_15856\143316924.py:1: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  (estimated_rotation,rmsd) = scipy.spatial.transform.Rotation.align_vectors(target_matched_xyz, source_matched_xyz)


(<scipy.spatial.transform._rotation.Rotation at 0x1c19c5637b0>,
 1.6220265998910222)

In [13]:
# match_targets.shape == match_sources.shape
np.linalg.norm(estimated_rotation.apply(match_sources) - match_targets)

22.87964538631396

In [14]:
# n_matched = min(target_fpfh_features.data.shape[1], source_fpfh_features.data.shape[1])

# n_target_f = target_fpfh_features.data.shape[1]
# n_source_f = source_fpfh_features.data.shape[1]

# p_over_norm = False

# min_disparity = np.inf
# min_target_sample = np.zeros((3,))
# min_source_sample = np.zeros((3,))
# # for i in range(10000):
# target_random_sample = np.random.choice(np.arange(n_target_f), (3,), replace=False)
# source_random_sample = np.random.choice(np.arange(n_source_f), (3,), replace=False)
# if p_over_norm:
#     mtx1, mtx2, disparity = procrustes(
#         target_fpfh_features.data[:,target_random_sample], 
#         source_fpfh_features.data[:,source_random_sample])
# else:
#     disparity = np.linalg.norm(
#         target_fpfh_features.data[:,target_random_sample] - 
#         source_fpfh_features.data[:,source_random_sample])
# if disparity < min_disparity:
#     min_disparity = disparity
#     min_target_sample = target_random_sample
#     min_source_sample = source_random_sample

# print(min_disparity, min_target_sample, min_source_sample)

In [15]:
target_matched_xyz = target_points[min_target_sample] - target_centroid
source_matched_xyz = source_points[min_source_sample] - source_centroid

NameError: name 'min_target_sample' is not defined

In [ ]:
x = np.array([[0, 2], [1, 1], [2, 0]]).T
x.shape, target_points[target_random_sample], 

np.append(target_points[target_random_sample], source_points[source_random_sample],axis=0).shape, x.shape
source_points.shape

(2154, 3)

In [ ]:
# Cpq = target_matched_xyz @ source_matched_xyz.T
# U,S,V = np.linalg.svd(Cpq)
# U @ V

In [ ]:
(estimated_rotation,rmsd) = scipy.spatial.transform.Rotation.align_vectors(target_matched_xyz, source_matched_xyz)
estimated_rotation,rmsd

(<scipy.spatial.transform._rotation.Rotation at 0x1943ac42990>,
 1.0179907252439597)

In [ ]:

estimated_rotation.apply(source_matched_xyz),target_matched_xyz


(array([[-0.48978131, -0.64845683,  0.13119908],
        [-0.22432958, -0.35277494,  0.03728706],
        [-0.18791448,  0.48169804, -0.12840193]]),
 array([[-0.67780466, -0.27464229,  0.02694767],
        [ 0.06418354,  0.17066639, -0.02039324],
        [-0.88154976,  0.55336459, -0.18775678]]))

In [ ]:
target_centered = target_points - target_centroid
source_centered = source_points - source_centroid
source_centered_transformed = estimated_rotation.apply(source_centered)

In [ ]:
disparity = np.linalg.norm(source_centered_transformed - target_centered)

ValueError: operands could not be broadcast together with shapes (3237,3) (2446,3) 

In [ ]:
# pcd1 = o3d.geometry.PointCloud()
# pcd1.points = o3d.utility.Vector3dVector(source_centered_transformed)
# pcd2 = o3d.geometry.PointCloud()
# pcd2.points = o3d.utility.Vector3dVector(target_centered)
# draw_registrations(pcd1, pcd2, recolor=True)

In [ ]:
# h = mapping_points.T @ true_points
# u, s, vt = np.linalg.svd(h)
# v = vt.T
# d = np.linalg.det(v @ u.T)
# e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])
# r = v @ e @ u.T

## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.